## 7.1 문자열을 날짜로 변환하기
날짜와 시간을 나타내는 문자열 벡터를 시계열 데이터로 변환하고 싶습니다.

판다스의 to_datetime 함수를 사용하고 format 매개변수에서 날짜와 시간 포맷을 지정합니다.

In [2]:
import numpy as np
import pandas as pd

date_strings = np.array(['03-04-2005 11:35 PM',
                         '23-05-2010 12:01 AM',
                         '04-09-2009 09:09 PM'])

# Timestamp 객체로 바꿉니다.
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

오류 처리를 위해 errors 매개변수를 추가할 수도 있습니다.

In [3]:
# datetime으로 바꿉니다.
[pd.to_datetime(date, format="%d-%m-%Y %I:%M %p", errors ="ignore") for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

errors="coerce"는 문제가 발생해도 에러를 일으키지 않지만 대신 에러가 난 값을 NaT(즉 누락된 값)으로 설정합니다.

날짜와 시간이 문자열일 때 이를 파이썬이 인식할 수 있는 데이터 타입으로 바꾸어야 합니다. 문자열을 datetime으로 바꿀 수 있는 파이썬 도구가 많이 있지만 다른 레시피에서 판다스를 사용하는 것에 맞추어 to_datetime으로 변환을 수행했습니다. 날짜와 시간 문자열을 다룰 때 어려운 점은 문자열의 포맷이 데이터 소스마다 크게 다르다는 것입니다. 예를 들어 어떤 날짜 벡터는 March 23rd, 2015를 03-23-15로 나타낼 수 있고 다른 경우는 3|23|2015일 수 있습니다. format 매개변수를 사용하여 적합한 문자열의 포맷을 지정해야 합니다. 다음은 많이 사용하는 날짜와 시간 포맷 코드입니다.

|코드|설명                         | 예 | 
|----|:----------------------------|:--:|
|%Y  |전체 연도                    |2001|
|%m  |0으로 시작하는 월            |04  |
|%d  |0으로 시작하는 일            |09  |
|%I  |0으로 시작하는 시간(12시간제)|02  |
|%p  |AM 또는 PM                   |AM  |
|%M  |0으로 시작하는 분            |05  |
|%S  |0으로 시작하는 초            |09  |

NaT(Not a Time)는 판다스의 datetime 데이터 타입에서 누락된 값을 의미합니다. to_datetime 함수의 errors 매개변수의 기본값은 'raise'로 날짜 포맷에 문제가 있을 때 예외를 발생시킵니다. 또 다른 옵션으로 'ignore'는 예외를 발생시키거나 NaT를 반환하는 대신 원본 문자열을 그대로 반환합니다.

해결에서처럼 리스트 컴프리헨션을 사용하지 않고 to_datetime 함수에 date_strings 리스트를 그대로 전달할 수 있습니다. 또한 format 매개변수를 지정하지 않아도 가능한 날짜 포맷을 추측하여 변환해 줍니다.

In [4]:
pd.to_datetime(date_strings)

DatetimeIndex(['2005-03-04 23:35:00', '2010-05-23 00:01:00',
               '2009-04-09 21:09:00'],
              dtype='datetime64[ns]', freq=None)

## 7.2 시간대 다루기
시계열 데이터에서 시간대 정보를 추가하거나 바꾸고 싶습니다.

특별히 지정하지 않으면 판다스 객체에는 시간대가 없습니다. 객체를 만들 때 tz 매개변수를 사용하여 시간대를 추가할 수 있습니다.

In [5]:
import pandas as pd

# datetime을 만듭니다.
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

tz_localize 메서드를 사용해 이전에 만든 datetime에 시간대를 추가할 수 있습니다.

In [7]:
# datetime을 만듭니다.
date = pd.Timestamp('2017-05-01 06:00:00')

# 시간대를 지정합니다.
date_in_london = date.tz_localize('Europe/London')

# datetime을 확인합니다.
date_in_london

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

다른 시간대로 변환할 수도 있습니다.

In [8]:
# 시간대를 바꿉니다. 
date_in_london.tz_convert('Africa/Abidjan')

Timestamp('2017-05-01 05:00:00+0000', tz='Africa/Abidjan')

마지막으로 판다스의 Series 객체는 모든 원소에 tz_localize와 tz_convert를 적용합니다.

In [9]:
# 세 개의 날짜를 만듭니다.
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='M'))

# 시간대를 지정합니다.
dates.dt.tz_localize('Africa/Abidjan')

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

판다스는 두 종류의 시간대 문자열을 지원합니다. 하지만 pytz 라이브러리의 문자열을 사용하는 것이 좋습니다. all_timezones을 임포트하여 전체 시간대 문자열을 확인할 수 있습니다.

In [10]:
from pytz import all_timezones

all_timezones[0:2]

['Africa/Abidjan', 'Africa/Accra']

'dateutil/'로 시작하여 dateutil 문자열을 사용할 수 있습니다.

In [12]:
dates.dt.tz_localize('dateutil/Asia/Seoul')

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, tzfile('ROK')]

pytz의 객체를 직접 전달할 수도 있습니다.

In [14]:
import pytz

tz = pytz.timezone('Asia/Seoul')
dates.dt.tz_localize(tz)

0   2002-02-28 00:00:00+09:00
1   2002-03-31 00:00:00+09:00
2   2002-04-30 00:00:00+09:00
dtype: datetime64[ns, Asia/Seoul]

## 7.3 날짜와 시간 선택하기
날짜 벡터에서 하나 이상의 원소를 선택하고 싶습니다.

시작과 마지막 날짜를 사용해 불리언 조건을 만듭니다.

In [15]:
import pandas as pd

dataframe = pd.DataFrame()

dataframe['date'] = pd.date_range('1/1/2001', periods=100000, freq='H')

dataframe[(dataframe['date'] > '2002-1-1 01:00:00') & 
          (dataframe['date'] <= '2002-1-1 04:00:00')]

,date
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


또는 날짜 열을 데이터프레임의 인덱스로 지정하고 loc를 사용해 슬라이싱할 수 있습니다.

In [16]:
dataframe = dataframe.set_index(dataframe['date'])

dataframe.loc['2002-1-1 01:00:00':'2002-1-1 04:00:00']

,date
date,
2002-01-01 01:00:00,2002-01-01 01:00:00
2002-01-01 02:00:00,2002-01-01 02:00:00
2002-01-01 03:00:00,2002-01-01 03:00:00
2002-01-01 04:00:00,2002-01-01 04:00:00


불리언 조건을 사용할지 인덱스 슬라이싱을 사용할지는 상황에 따라 다릅니다. 복잡한 시계열 데이터를 다루어야 한다면 날짜 열을 데이터프레임의 인덱스로 지정할 만 합니다. 간단한 데이터 랭글링이라면 불리언 조건이 더 쉽습니다.

## 7.4 날짜 데이터를 여러 특성으로 나누기
날짜와 시간의 열을 가지고 년, 월, 일, 시, 분에 해당하는 특성을 만들고 싶습니다.

Series.dt의 시간 속성을 사용합니다.

In [1]:
import pandas as pd

dataframe = pd.DataFrame()

dataframe['date'] = pd.date_range('1/1/2001', periods=150, freq='W')

# 년, 월, 일, 시, 분에 대한 특성을 만듭니다.
dataframe['year'] = dataframe['date'].dt.year
dataframe['month'] = dataframe['date'].dt.month
dataframe['day'] = dataframe['date'].dt.day
dataframe['hour'] = dataframe['date'].dt.hour
dataframe['minute'] = dataframe['date'].dt.minute

dataframe.head(3)

,date,year,month,day,hour,minute
0,2001-01-07,2001,1,7,0,0
1,2001-01-14,2001,1,14,0,0
2,2001-01-21,2001,1,21,0,0


이따금 날짜 열을 개별 항목으로 나누는 것이 유용합니다. 예를 들어, 샘플의 연도만 가지는 특성이 필요하거나 연도와 상관없이 샘플의 월만 비교해야 할 수 있습니다.

## 7.5 날짜 간의 차이를 계산하기
샘플에 두 개의 datetime 특성이 있을 때 이 특성 사이의 시간을 계산하고 싶습니다.

판다스를 사용하여 두 날짜 특성을 뺍니다.

In [2]:
import pandas as pd

dataframe = pd.DataFrame()

dataframe['Arrived'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-04-2017')]
dataframe['Left'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-06-2017')]

dataframe['Left'] - dataframe['Arrived']

0   0 days
1   2 days
dtype: timedelta64[ns]

days 출력을 삭제하고 수치값만 남기고 싶을 때가 종종 있습니다.

In [3]:
pd.Series(delta.days for delta in (dataframe['Left'] - dataframe['Arrived']))

0    0
1    2
dtype: int64

두 지점 사이의 시간 변화를 기록한 특성이 필요할 때가 있습니다. 예를 들어 호텔의 체크인 시간과 체크아웃 시간을 알고 있을 때 고객이 머문 시간이 특성으로 필요할 수 있습니다. 판다스의 TimeDelta 데이터 타입을 사용하면 이런 계산을 손쉽게 처리할 수 있습니다.

## 7.6 요일을 인코딩하기
날짜 벡터에서 각 날짜의 요일을 알고 싶습니다.

판다스 Series.dt의 day_name 메서드를 사용합니다.

In [4]:
import pandas as pd

dates = pd.Series(pd.date_range("2/2/2002", periods=3, freq="M"))

dates.dt.day_name()

0    Thursday
1      Sunday
2     Tuesday
dtype: object

머신러닝의 특성으로 사용하기 좋도록 수치형 값을 출력하고 싶다면 weekday를 사용하여 요일을 정수로 나타낼 수 있습니다(월요일이 0입니다).

In [7]:
dates.dt.weekday

0    3
1    6
2    1
dtype: int64

요일을 알면 유용할 때가 있습니다. 예를 들면 지난 3년간 일요일에 일어난 전체 판매량을 비교할 수 있습니다. 판다스는 요일 정보를 가진 특성 벡터를 쉽게 만들어줍니다.

## 7.7 시차 특성 만들기
n기간 만큼 차이가 나는 시차 특성(lagged feature)을 만들고 싶습니다.

판다스의 shift를 사용합니다.

In [8]:
import pandas as pd

dataframe = pd.DataFrame()

dataframe['dates'] = pd.date_range("1/1/2001", periods=5, freq="D")
dataframe['stock_price'] = [1.1,2.2,3.3,4.4,5.5]

# 한 행 뒤의 값을 가져옵니다.
dataframe['previous_days_stock_price'] = dataframe['stock_price'].shift(1)

dataframe

,dates,stock_price,previous_days_stock_price
0,2001-01-01,1.1,NaN
1,2001-01-02,2.2,1.1
2,2001-01-03,3.3,2.2
3,2001-01-04,4.4,3.3
4,2001-01-05,5.5,4.4


데이터가 일정한 시간 간격으로 생성된 경우가 아주 많습니다(예를 들어 일마다, 한 시간마다, 세 시간마다). 이런 데이터셋에서 과거 값을 사용하여 예측을 만들어야 합니다(이를 시차 특성을 사용한다고 말합니다).예를 들면 하루 전 주식 가격을 사용해 오늘 가격을 예측하는 경우입니다. 판다스의 shift 메서드를 사용하여 한 행 뒤의 값을 담은 새로운 특성을 만들 수 있습니다. 

## 7.8 이동 시간 윈도 사용하기
시계열 데이터에서 일정 시간 간격으로 통계를 계산하고 싶습니다.

In [9]:
import pandas as pd

# datetime을 만듭니다.
time_index = pd.date_range('01/01/2010', periods=5, freq="M")

# 데이터프레임을 만들고 인덱스를 설정합니다.
dataframe = pd.DataFrame(index=time_index)

# 특성을 만듭니다.
dataframe['Stock_Price'] = [1,2,3,4,5]

# 이동 평균을 계산합니다.
dataframe.rolling(window=2).mean()

,Stock_Price
2010-01-31,NaN
2010-02-28,1.5
2010-03-31,2.5
2010-04-30,3.5
2010-05-31,4.5


이동 시간 윈도(rolling time window 또는 moving time window)는 간단한 개념이지만 처음에는 이해하기 어려울 수 있습니다. 월간 주식 가격 데이터가 있다고 가정해보죠. 몇 달을 하나의 시간 윈도로 설정하고 윈도를 이동하면서 윈도 안의 모든 샘플의 통계를 계산할 필요가 있습니다.

예를 들어 세 달을 시간 윈도로 정하면 다음과 같이 이동 평균을 계산합니다.

- mean (January, February, March)
- mean (February, March, April)
- mean (March, April, May)

다른 식으로 말하면 세 달 크기의 시간 윈도가 각 단계마다 윈도의 평균을 계산하면서 샘플 위를 이동합니다.

판다스의 rolling 메서드는 window 매개변수에서 윈도 크기를 지정할 수 있습니다. 최댓값(max()), 평균(mean()), 샘플 개수(count()), 상관관계(corr())와 같은 통계를 간편하게 계산할 수 있습니다.

시간 윈도의 평균은 단기간의 급한 변화를 완화시키기 떄문에 시계열 데이터를 부드럽게 만들기 위해 이동 평균이 종종 사용됩니다.

최근 항목에 높은 가중치를 두지만 전체 기간에 대한 통계를 계산하는 지수 이동 윈도(exponential moving window)도 많이 사용하는 방법입니다. 판다스에서는 ewm 메서드를 사용하여 계산할 수 있습니다.

In [10]:
dataframe.ewm(alpha=0.5).mean()

,Stock_Price
2010-01-31,1.000000
2010-02-28,1.666667
2010-03-31,2.428571
2010-04-30,3.266667
2010-05-31,4.161290


## 7.9 시계열 데이터에서 누락된 값 다루기
시계열 데이터에 누락된 값이 있습니다.

이전에 누락된 값을 다루는 전략 외에 시계열 데이터에서는 보간(interpolation) 방법을 사용하여 누락된 값으로 생긴 간격을 채울 수 있습니다.

In [12]:
import pandas as pd
import numpy as np

# datetime을 만듭니다.
time_index = pd.date_range('01/01/2010', periods=5, freq="M")

# 데이터프레임을 만들고 인덱스를 설정합니다.
dataframe = pd.DataFrame(index=time_index)

# 누락된 값이 있는 특성을 만듭니다.
dataframe['Sales'] = [1.0,2.0,np.nan,np.nan,5.0]

# 누락된 값을 보간합니다.
dataframe.interpolate()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


또는 누락된 값을 이전에 등장한 마지막 값으로 대체할 수 있습니다(즉, 앞쪽의 빈 값을 채웁니다).

In [13]:
# 앞쪽으로 채우기(Forward-fill)
dataframe.ffill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


누락된 값을 그 이후에 등장한 최초의 값으로 대체할 수도 있습니다(즉, 뒤쪽의 빈 값을 채웁니다).

In [14]:
# 뒤쪽으로 채우기(Back-fill)
dataframe.bfill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


보간은 누락된 값의 양쪽 경계를 잇는 직선이나 곡선을 사용하여 적절한 값을 예측함으로써 비어 있는 간극을 채우는 기법입니다. 특히 시간 간격이 일정하고, 데이터가 노이즈로 인한 변동이 심하지 않고 누락된 값으로 인한 빈 간극이 작을 때 보간 방식이 유용합니다. 예를 들어 해결에서 두 개의 누락된 값이 만든 간격의 경계값은 2.0과 5.0입니다. 2.0에서 5.0까지 직선을 그으면 중간에 누락된 두 개의 값을 3.0과 4.0으로 추측할 수 있습니다.

두 포인트 사이의 직선이 비선형이라고 가정하면 interpolate의 method 매개변수를 사용해 다른 보간 방법을 지정할 수 있습니다.

In [15]:
# 누락된 값을 보간하기
dataframe.interpolate(method="quadratic")

,Sales
2010-01-31,1.000000
2010-02-28,2.000000
2010-03-31,3.059808
2010-04-30,4.038069
2010-05-31,5.000000


마지막으로 누락된 값의 간격이 커서 전체 간격을 보간하는 것이 좋지 않을 때가 있습니다. 이런 경우에 limit 매개변수를 사용하여 보간 값의 개수를 제한하고 limit_direction 매개변수에서 마지막 데이터로 앞쪽 방향으로 보간할지 그 반대로 할 지 지정할 수 있습니다.

In [16]:
dataframe.interpolate(limit=1, limit_direction='forward')

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,NaN
2010-05-31,5.0


뒤쪽으로 채우기와 앞쪽으로 채우기는 단순한 보간의 한 형태로 생각할 수 있습니다. 알고 있는 값에서 수평한 직선을 긋고 이를 사용해 누락된 값을 채우는 방식입니다. 보간에 비해 뒤쪽으로 채우기와 앞쪽으로 채우기의 (작은) 장점은 누락된 값의 앞뒤 값을 모두 알 필요가 없다는 점입니다.